In [1]:
import pandas as pd
import itertools
import numpy as np
from scipy.stats import qmc
import matplotlib.pyplot as plt
import csv
import time
import sys
from phasepy import mixture, component, virialgamma, preos
from phasepy.equilibrium import tpd_min, tpd_minimas, lle_init, sle, slle

In [2]:
methanol = component(name='methanol', Tc=512.5, Pc=80.84, Zc=0.222, Vc=117, GC={'CH3OH':1})
ethanol = component(name='ethanol', Tc=514, Pc=61.37, Zc=0.241, Vc=168, GC={'CH3':1, 'CH2':1, 'OH':1})
_2propanol = component(name='2-propanol', Tc=508.3, Pc=47.65, Zc=0.25, Vc=222, GC={'CH3':2, 'CH':1, 'OH':1})
acetone = component(name='acetone', Tc=508.1, Pc=47, Zc=0.237, Vc=213, GC={'CH3CO':1, 'CH3':1})
mibk = component(name='mibk', Tc=574.6, Pc=32.7, Zc=0.256, Vc=317, GC={'CH3CO':1, 'CH3':2, 'CH2':1, 'CH':1})
ethac = component(name='ethyl-acetate', Tc=523.3, Pc=38.8, Zc=0.255, Vc=286, GC={'CH3COO':1, 'CH3':1, 'CH2':1})
chloroform = component(name='chloroform', Tc=536.4, Pc=54.72, Zc=0.293, Vc=239, GC={'CHCL3':1})
toluene = component(name='toluene', Tc=591.75, Pc=41.08, Zc=0.264, Vc=316, GC={'ACH':5, 'ACCH3':1})
water = component(name='water', Tc=647.096, Pc=220.64, Zc=0.229, Vc=55.9472,GC={'H2O':1})
ibuprofen = component(name='ibuprofen', Tc=765, Pc=29.8, Zc=0.313, Vc=668,
                      GC={'CH3':3, 'CH':1, 'ACH':4, 'ACCH2':1, 'ACCH':1, 'COOH':1},
                      dHf=25500, Tf=347.15)

In [3]:
component_map = {"methanol" : methanol, "ethanol" : ethanol, "2-propanol" : _2propanol, "acetone" : acetone,
                 "mibk" : mibk, "ethyl-acetate": ethac, "chloroform" : chloroform, "toluene": toluene,
                 "water" : water}

In [4]:
component_list = [methanol,ethanol,_2propanol,acetone,mibk,ethac,chloroform,toluene,water]
binary_mix = itertools.combinations(component_list, 2)

In [5]:
def return_eos(c1,c2):
    mix_func = mixture(c1,c2)
    mix_func += ibuprofen
    mix_func.original_unifac()
    eos = virialgamma(mix_func, virialmodel='ideal_gas', actmodel='original_unifac')
    return eos

In [6]:
%%time
eos_dict = {}
counter=0
for i in binary_mix:
    eos = return_eos(i[0],i[1])
    eos_dict[counter] = eos
    counter+=1

CPU times: total: 11.6 s
Wall time: 17.8 s


In [7]:
def return_phases(b,t):
    liq_flag = 0
    liq_liq_flag = 0
    sol_liq_flag = 0
    sol_liq_liq_flag = 0
    if (b[0] == 1 and b[1] == 0 and b[2] == 0 and t[0] == 0 and t[1] >= 0) or (b[0] == 0 and b[1] == 1 and b[2] == 0 and t[0] == 0 and t[1] >= 0) or (b[0] == 1 and b[1] == 0 and b[2] == 0 and t[0] > 0 and t[1] > 0) or (b[0] == 0 and b[1] == 1 and b[2] == 0 and t[0] > 0 and t[1] > 0):
            liq_flag = 1
    elif (b[0] >= 0 and b[1] >= 0 and b[2] == 0 and t[0] == 0 and t[1] >= 0):
            liq_liq_flag = 1
    elif (b[0] > 0 and b[1]==0 and b[2]>0 and t[0]==0 and t[1]==0) or (b[0] == 0 and b[1]>0 and b[2]>0 and t[0]==0 and t[1]==0) or (b[0] > 0 and b[1]==0 and b[2]>0 and t[0]>0 and t[1]==0) or (b[0] == 0 and b[1]>0 and b[2]>0 and t[0]>0 and t[1]==0):
            sol_liq_flag = 1
    elif (b[0] > 0 and b[1] > 0 and b[2] > 0 and t[0] == 0 and t[1] == 0):
            sol_liq_liq_flag = 1
    return liq_flag, liq_liq_flag, sol_liq_flag, sol_liq_liq_flag

In [8]:
df_part = pd.read_csv('Case1.csv', nrows=512)
z1 = df_part['x1']
z2 = df_part['x2']
z3 = 1 - z1 - z2
z = np.column_stack((z1, z2, z3))
T = df_part['T']

In [9]:
st = time.time() #start timer
nmin = 3
P = 1.01325
ismisc = []
for i in eos_dict:
    for j in range(512):
        tpd_m = slle(z[j], T[j], P, eos_dict[i], solid_phases_index=[2], full_output=True, accelerate_every=20)
        theta = tpd_m.tetha
        betas = tpd_m.beta
        liq_flag, liq_liq_flag, sol_liq_flag, sol_liq_liq_flag = return_phases(betas,theta)
        if liq_flag == 1 or sol_liq_flag == 1:
            ismisc.append(1)
        elif liq_liq_flag == 1 or sol_liq_liq_flag == 1:
            ismisc.append(0)
        else:
            print(z[j],T[j])
            print(tpd_m)
            ismisc.append(2)

et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds') #execution time

c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:16: RuntimeWarning: overflow encountered in exp
  Kexp = K.T*np.exp(tetha)
c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:16: RuntimeWarning: invalid value encountered in multiply
  Kexp = K.T*np.exp(tetha)
c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:22: RuntimeWarning: invalid value encountered in multiply
  f2 = (K1.T*Xref).sum(axis=1)
c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:35: RuntimeWarning: invalid value encountered in multiply
  ter1 = Xref/sum1 * K1.T
c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:38: RuntimeWarning: invalid value encountered in multiply
  dft = Id * (Xref*Kexp.T).sum(axis=1)
c:\users\tk3016\phasepy\phasepy\equilibrium\solidequilibria.py:172: RuntimeWarning: invalid value encountered in multiply
  X[1:(n_fluid)] = (Xref*Kexp.T)[:(n_fluid-1)]


[0.44727861 0.45772789 0.0949935 ] 300
            T: 300
            P: 1.01325
  error_outer: nan
  error_inner: nan
         iter: 41
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan, nan],
       [nan, nan, nan]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.50111888 0.47877134 0.02010978] 300
            T: 300
            P: 1.01325
  error_outer: nan
  error_inner: nan
         iter: 6
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan, nan],
       [nan, nan, nan]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])


c:\users\tk3016\phasepy\phasepy\equilibrium\solidequilibria.py:212: RuntimeWarning: overflow encountered in exp
  K = np.exp(lnK)
c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:36: RuntimeWarning: invalid value encountered in matmul
  dfb = - ter1@K1
c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:40: RuntimeWarning: invalid value encountered in matmul
  dft -= ter1 @ ter2


[0.08579615 0.53076801 0.38343583] 300
            T: 300
            P: 1.01325
  error_outer: nan
  error_inner: nan
         iter: 82
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan, nan],
       [nan, nan, nan]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.04904503 0.47464404 0.47631093] 300
            T: 300
            P: 1.01325
  error_outer: nan
  error_inner: nan
         iter: 41
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan, nan],
       [nan, nan, nan]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])


c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:19: RuntimeWarning: divide by zero encountered in divide
  Xref = Z/sum1
c:\users\tk3016\phasepy\phasepy\equilibrium\solidequilibria.py:174: RuntimeWarning: invalid value encountered in divide
  X = (X.T/X.sum(axis=1)).T


[0.0449876  0.01694294 0.93806946] 300
            T: 300
            P: 1.01325
  error_outer: 1.3367884204904339e-14
  error_inner: 4392289123862.5537
         iter: 12
         beta: array([0., 0., 1.])
        tetha: array([0.00014134, 0.        ])
      X_fluid: array([[7.26420332e-01, 2.73579668e-01, 1.76508399e-14],
       [7.26420345e-01, 2.73579655e-01, 1.76508426e-14]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
Execution time: 1622.1053488254547 seconds


In [10]:
print(len(ismisc))

18432


In [11]:
df = pd.read_csv('Case1.csv')
df['ismisc'] = ismisc
df2 = df[df.ismisc != 2]
df2

,Unnamed: 0,Solvent 1,Solvent 2,q1,q2,r1,r2,x1,x2,T,ismisc
0,0,methanol,ethanol,1.432,2.588,1.4311,2.5755,0.312852,0.132230,300,1
1,1,methanol,ethanol,1.432,2.588,1.4311,2.5755,0.362349,0.564846,300,1
2,2,methanol,ethanol,1.432,2.588,1.4311,2.5755,0.507916,0.181390,300,1
3,3,methanol,ethanol,1.432,2.588,1.4311,2.5755,0.044826,0.656056,300,1
4,4,methanol,ethanol,1.432,2.588,1.4311,2.5755,0.255614,0.554332,300,1
...,...,...,...,...,...,...,...,...,...,...,...
18427,18427,toluene,water,2.968,1.400,3.9228,0.9200,0.183227,0.484442,300,0
18428,18428,toluene,water,2.968,1.400,3.9228,0.9200,0.085592,0.604176,300,0
18429,18429,toluene,water,2.968,1.400,3.9228,0.9200,0.379976,0.273439,300,0
18430,18430,toluene,water,2.968,1.400,3.9228,0.9200,0.489890,0.177571,300,0


In [12]:
df2.to_csv('Case1_tpd_calc_slle.csv')